# ЛР 5
## Задания:
1. Провести классификацию найденного датасета, методами решающего дерева и случайного леса . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

In [3]:
import pandas as pd
import numpy as np

df = pd.read_csv('train.csv', index_col=0)  

In [4]:
df

,MonthlyCharges,TotalCharges,SubscriptionType,PaymentMethod,PaperlessBilling,ContentType,MultiDeviceAccess,DeviceRegistered,ViewingHoursPerWeek,AverageViewingDuration,ContentDownloadsPerMonth,GenrePreference,UserRating,SupportTicketsPerMonth,Gender,WatchlistSize,ParentalControl,SubtitlesEnabled,CustomerID,Churn
AccountAge,,,,,,,,,,,,,,,,,,,,
20,11.055215,221.104302,Premium,Mailed check,No,Both,No,Mobile,36.758104,63.531377,10,Sci-Fi,2.176498,4,Male,3,No,No,CB6SXPNVZA,0
57,5.175208,294.986882,Basic,Credit card,Yes,Movies,No,Tablet,32.450568,25.725595,18,Action,3.478632,8,Male,23,No,Yes,S7R2G87O09,0
73,12.106657,883.785952,Basic,Mailed check,Yes,Movies,No,Computer,7.395160,57.364061,23,Fantasy,4.238824,6,Male,1,Yes,Yes,EASDC20BDT,0
32,7.263743,232.439774,Basic,Electronic check,No,TV Shows,No,Tablet,27.960389,131.537507,30,Drama,4.276013,2,Male,24,Yes,Yes,NPF69NT69N,0
57,16.953078,966.325422,Premium,Electronic check,Yes,TV Shows,No,TV,20.083397,45.356653,20,Comedy,3.616170,4,Female,0,No,No,4LGYPK7VOL,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,12.041577,216.748394,Standard,Bank transfer,No,Both,No,Mobile,15.720485,123.854169,29,Sci-Fi,3.960361,4,Female,18,Yes,No,NKNHL3SRNX,0
57,10.104155,575.936861,Basic,Credit card,Yes,Both,Yes,TV,23.795246,161.325894,42,Comedy,3.910286,2,Male,22,No,Yes,24794SG5S1,0
56,12.205436,683.504390,Premium,Credit card,No,TV Shows,Yes,Mobile,1.332610,140.223702,45,Fantasy,4.571555,1,Male,4,Yes,Yes,VYGN9JRW4P,0


Замена текстовых категориальных признаков на числовые

In [5]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
df['SubscriptionType'] = labelencoder.fit_transform(df['SubscriptionType'])
df['PaymentMethod'] =  labelencoder.fit_transform(df['PaymentMethod'])
df['PaperlessBilling'] =  labelencoder.fit_transform(df['PaperlessBilling'])
df['ContentType'] =  labelencoder.fit_transform(df['ContentType'])
df['MultiDeviceAccess'] =  labelencoder.fit_transform(df['MultiDeviceAccess'])
df['DeviceRegistered'] =  labelencoder.fit_transform(df['DeviceRegistered'])
df['GenrePreference'] =  labelencoder.fit_transform(df['GenrePreference'])
df['Gender'] =  labelencoder.fit_transform(df['Gender'])
df['ParentalControl'] =  labelencoder.fit_transform(df['ParentalControl'])
df['SubtitlesEnabled'] =  labelencoder.fit_transform(df['SubtitlesEnabled'])
df['CustomerID'] =  labelencoder.fit_transform(df['CustomerID'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100029 entries, 20 to 77
Data columns (total 20 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   MonthlyCharges            100029 non-null  float64
 1   TotalCharges              100029 non-null  float64
 2   SubscriptionType          100029 non-null  int32  
 3   PaymentMethod             100029 non-null  int32  
 4   PaperlessBilling          100029 non-null  int32  
 5   ContentType               100029 non-null  int32  
 6   MultiDeviceAccess         100029 non-null  int32  
 7   DeviceRegistered          100029 non-null  int32  
 8   ViewingHoursPerWeek       100029 non-null  float64
 9   AverageViewingDuration    100029 non-null  float64
 10  ContentDownloadsPerMonth  100029 non-null  int64  
 11  GenrePreference           100029 non-null  int32  
 12  UserRating                100029 non-null  float64
 13  SupportTicketsPerMonth    100029 non-null  int64  
 

Разбиение на train/test и обучение модели с разными параметрами (Решающее дерево)

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from imblearn.under_sampling import RandomUnderSampler
from sklearn.tree import DecisionTreeClassifier
import matplotlib.pyplot as plt


X = df.drop(columns=['Churn'])
Y = df['Churn']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=27)


under_sampler = RandomUnderSampler(random_state=27)
X_resampled, Y_resampled = under_sampler.fit_resample(X_train, Y_train)

decision_tree = DecisionTreeClassifier()
dt_params = {
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

dt_grid_search = GridSearchCV(decision_tree, dt_params, cv=5)
dt_grid_search.fit(X_resampled, Y_resampled)

dt_best_params = dt_grid_search.best_params_
dt_best_max_depth = dt_best_params['max_depth']
dt_best_min_ss = dt_best_params['min_samples_split']
dt_best_min_sl = dt_best_params['min_samples_leaf']

print(f"Лучшие параметры для решающего дерева: Max Depth: {dt_best_max_depth}, Min samples split: {dt_best_min_ss}, Min samples leaf: {dt_best_min_sl}")

Лучшие параметры для решающего дерева: Max Depth: 5, Min samples split: 2, Min samples leaf: 1


Оценка точности и отчёт:

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

decision_tree_model = DecisionTreeClassifier(max_depth = dt_best_max_depth, min_samples_leaf =  dt_best_min_sl, min_samples_split = dt_best_min_ss)
decision_tree_model.fit(X_resampled, Y_resampled)

Y_pred = decision_tree_model.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
print(f"Точность модели: {accuracy}")

report = classification_report(Y_test, Y_pred, zero_division=1)
print(report)

Точность модели: 0.5859575460695124
              precision    recall  f1-score   support

           0       0.90      0.55      0.69     24522
           1       0.27      0.73      0.39      5487

    accuracy                           0.59     30009
   macro avg       0.58      0.64      0.54     30009
weighted avg       0.78      0.59      0.63     30009



Разбиение на train/test и обучение модели с разными параметрами (Случайный лес)

In [9]:
from sklearn.ensemble import RandomForestClassifier

X = df.drop(columns=['Churn'])
Y = df['Churn']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=27)


under_sampler = RandomUnderSampler(random_state=27)
X_resampled, Y_resampled = under_sampler.fit_resample(X_train, Y_train)

random_forest = RandomForestClassifier()
rf_params = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [None, 5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf_grid_search = GridSearchCV(random_forest, rf_params, cv=5)
rf_grid_search.fit(X_resampled, Y_resampled)

rf_best_params = rf_grid_search.best_params_
rf_best_n_estimators = rf_best_params['n_estimators']
rf_best_max_depth = rf_best_params['max_depth']
rf_best_min_ss = rf_best_params['min_samples_split']
rf_best_min_sl = rf_best_params['min_samples_leaf']

print(f"Лучшие параметры для случайного леса: N estimators: {rf_best_n_estimators}, Max Depth: {dt_best_max_depth}, Min samples split: {dt_best_min_ss}, Min samples leaf: {dt_best_min_sl}")

Лучшие параметры для случайного леса: N estimators: 200, Max Depth: 5, Min samples split: 2, Min samples leaf: 1


Оценка точности и отчёт:

In [10]:
random_forest_model = RandomForestClassifier(n_estimators = rf_best_n_estimators, max_depth = rf_best_max_depth, min_samples_leaf =  rf_best_min_sl, min_samples_split = rf_best_min_ss)
random_forest_model.fit(X_resampled, Y_resampled)

Y_pred = random_forest_model.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
print(f"Точность модели: {accuracy}")

report = classification_report(Y_test, Y_pred, zero_division=1)
print(report)

Точность модели: 0.6694991502549236
              precision    recall  f1-score   support

           0       0.90      0.67      0.77     24522
           1       0.31      0.68      0.43      5487

    accuracy                           0.67     30009
   macro avg       0.61      0.68      0.60     30009
weighted avg       0.80      0.67      0.71     30009

